In [202]:
import re
from typing import List

class Calculator:
    def __init__(self):
        self.tokens = None

    def _tokenize(self, expression: str) -> None:
        pattern = r'-?[0-9]*\.?[0-9]+|[()+*\/-]' # Выделяем числа, скобки или функции
        tokens = re.findall(pattern, expression)
        self.tokens = tokens
        
    def _rpn_transformation(self) -> List[str]:
        "Функция перевод инвиксной записи в постфиксную польскую запись"
        priority = {'+': 1, '-': 1, '*': 2, '/': 2, '~': 3}
        output = []
        stack = []
        
        for idx, token in enumerate(self.tokens):
            pattern = r'^-?\d+(?:\.\d+)?$' # Выделяем вещественное число
            if re.match(pattern, token):
                output.append(token)
            elif token == '(':
                stack.append(token)
            elif token == ')':
                while stack and stack[-1] != '(':
                    output.append(stack.pop())
                stack.pop()
            elif token in priority:
                # Если минус унарный, используем ~ как оператор унарного минуса
                # также можно добавлять 0 перед -
                if token == '-' and (output == [] or
                    not(re.match(pattern, self.tokens[idx-1]))):
                    token = '~'
                while stack and stack[-1] in priority and priority[stack[-1]] >= priority[token]:
                    output.append(stack.pop())
                stack.append(token)
                
        while stack:
            output.append(stack.pop())
        
        return output

    def evaluate(self, expression: str) -> float:
        "Функция подсчёта результата выражения согласно постфиксной польской нотации"
        self._tokenize(expression)

        rpn = self._rpn_transformation()
        stack = []
        
        for token in rpn:
            if re.match(r'^-?\d+(?:\.\d+)?$', token):
                stack.append(float(token))
            elif token == '+':
                stack.append(stack.pop() + stack.pop())
            elif token == '-':
                a = stack.pop()
                b = stack.pop()
                stack.append(b - a)
            elif token == '*':
                stack.append(stack.pop() * stack.pop())
            elif token == '/':
                a = stack.pop()
                b = stack.pop()
                if a == 0:
                    raise ZeroDivisionError("Деление на ноль запрещено")
                stack.append(b / a)
            elif token == '~':
                last = 0 if stack == [] else stack.pop()
                stack.append(0 - last)
        return stack.pop()

calculator = Calculator()

In [193]:
from flask import Flask, jsonify, request
from Calculator import Calculator

app = Flask(__name__)

@app.route('/', methods=['POST'])
def calculate():
    expression = request.get_json()['expression']
    try:
        calculator = Calculator()
        result = calculator.evaluate(expression)
        return jsonify({'input': expression, 'result': result})
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Dec/2023 21:40:27] "POST / HTTP/1.1" 200 -


In [215]:
assert calculator.evaluate("5 + 3 * 2") == 11

assert calculator.evaluate("(2 + 4) * 3 + 5 / 2") == 20.5

assert calculator.evaluate("10 * (4 - 2) + 5") == 25

assert calculator.evaluate("4 + 2 * (6 - 3) / 2") == 7

assert calculator.evaluate("2 * -3") == -6

assert calculator.evaluate("1.5 * ((2 - 4 + 3) * 2) / 1.2") == 2.5

assert calculator.evaluate("2 * -(-5)") == 10

assert calculator.evaluate("5 + 2 / 4 - 1") == 4.5

assert calculator.evaluate("10 / (4 + 2 * 3)") == 1

assert calculator.evaluate("(4 - 2) / 5 + 3 * 2") == 6.4

assert calculator.evaluate("-4 + 2 * (6 - 1)") == 6

assert calculator.evaluate("-2 - -3 * 4") == 10

assert calculator.evaluate("(2 * 3 - 4) / (1 + 2)") == 0.6666666666666666